**Question 1: What is the role of filters and feature maps in Convolutional Neural Network (CNN)?**
#### **Answer**:
**Filters (Kernels):**  
- Small learnable matrices (e.g., 3×3 or 5×5) that slide over the input image.  
- Each filter detects a specific pattern such as edges, textures, or shapes.  
- During training, the network learns the optimal filter values to capture useful features.

**Feature Maps (Activation Maps):**  
- The output generated when a filter is convolved with the input.  
- Highlight *where* a particular feature (edge, corner, texture) appears in the image.  
- Stacking multiple feature maps from different filters forms the representation for the next layer.

> **In short:**  
> **Filters** = *what to look for*  
> **Feature Maps** = *where it is found*


#**Question 2: Explain the concepts of padding and stride in CNNs(Convolutional Neural Network). How do they affect the output dimensions of feature maps?**
#### **Answer**:
#### **Padding**
- **Definition:** Adding extra pixels (usually zeros) around the input image’s border before applying the convolution.
- **Purpose:**
  - Preserve spatial size after convolution.
  - Avoid losing edge information.
- **Common Types:**
  - **Valid (no padding):** No extra pixels. Output shrinks.
  - **Same (zero padding):** Pads so output size ≈ input size when stride = 1.

#### **Stride**
- **Definition:** The number of pixels the filter moves at each step horizontally and vertically.
- **Effects:**
  - Larger stride → skips more pixels → smaller output.
  - Stride of 1 → slides filter one pixel at a time → larger output.


###  Output Dimension Formula
$$
\text{Output Height} = \left\lfloor \frac{H - K + 2P}{S} \right\rfloor + 1
$$

$$
\text{Output Width} = \left\lfloor \frac{W - K + 2P}{S} \right\rfloor + 1
$$

Where:  
- $H, W$ = input height & width  
- $K$ = kernel size  
- $P$ = padding  
- $S$ = stride  



###  Quick Examples
1. **No padding, stride 1**  
   - Input: 32×32, Kernel: 3×3  
   - Output: (32 − 3 + 0)/1 + 1 = **30×30**

2. **Same padding, stride 1**  
   - Input: 32×32, Kernel: 3×3, Padding: 1  
   - Output: (32 − 3 + 2)/1 + 1 = **32×32**

3. **Padding 1, stride 2**  
   - Input: 32×32, Kernel: 3×3  
   - Output: ⌊(32 − 3 + 2)/2⌋ + 1 = **16×16**
   

#**Question 3: Define receptive field in the context of CNNs. Why is it important for deep architectures?**
#### **Answer**:
**Definition:**  
The **receptive field** is the region of the input image that influences a neuron’s activation.  
Example: a $3 \times 3$ kernel gives each neuron a $3 \times 3$ receptive field.

**Growth in Deep Networks:**  
With stacked layers, the effective receptive field enlarges:

$$
R_l = R_{l-1} + (k_l - 1)\!\times\!\prod_{i=1}^{l-1} s_i
$$

where $k_l$ = kernel size, $s_i$ = stride, $R_1 = k_1$.

**Importance:**  
- Captures broader context for detecting large objects.  
- Enables higher layers to recognize full shapes, not just edges.  
- Guides network design to ensure the final layers “see” enough of the input.


#**Question 4: Discuss how filter size and stride influence the number of parameters in a CNN.**
#### **Answer**:
**1️ Filter (Kernel) Size**
- Parameters in a conv layer depend on:
  $$
  \text{Params} = (K_h \times K_w \times C_{in}) \times C_{out} + C_{out}
  $$
  where  
  - $K_h, K_w$ = filter height & width  
  - $C_{in}$ = input channels  
  - $C_{out}$ = number of filters (output channels)  
- **Impact:** Larger filters ($K_h,K_w$) ⇒ **more parameters** because each filter contains more weights.

**2️ Stride**
- **Stride does *not* change the number of trainable parameters**, because filter dimensions and counts stay the same.
- It only changes the **output feature-map size**, thus affecting:
  - Memory usage
  - Computational cost (fewer output locations when stride is large).


**Summary Table**

| Hyperparameter | Parameters | Output Size |
|----------------|-----------|------------|
| **Filter size ↑** | ↑ (more weights) | Slightly smaller (unless padded) |
| **Stride ↑**    | No change | ↓ (fewer activations) |


#**Question 5: Compare and contrast different CNN-based architectures like LeNet, AlexNet, and VGG in terms of depth, filter sizes, and performance.**
#### **Answer**:
| Feature            | **LeNet-5 (1998)** | **AlexNet (2012)**          | **VGG-16/19 (2014)** |
|--------------------|--------------------|-----------------------------|----------------------|
| **Depth**         | ~5 learnable layers (2 conv + 3 FC) | 8 learnable layers (5 conv + 3 FC) | 16 or 19 learnable layers (13/16 conv + 3 FC) |
| **Filter Sizes**   | Mostly $5 \times 5$ | First layer $11 \times 11$, others $5 \times 5$ and $3 \times 3$ | Uniform $3 \times 3$ filters throughout |
| **Input Size**     | $32 \times 32$ grayscale | $224 \times 224$ RGB | $224 \times 224$ RGB |
| **Parameters**     | ~60K               | ~60M                        | ~138M (VGG-16)       |
| **Performance / Impact** | Pioneered CNNs for digit recognition (MNIST) | Won ImageNet 2012, introduced ReLU & dropout, GPU training | High accuracy on ImageNet; showed that deep networks with small filters work well |
| **Key Innovations** | Early CNN with subsampling | Overlapping max-pooling, data augmentation | Deep, uniform architecture for scalable depth |














#**Question 6: Using keras, build and train a simple CNN model on the MNIST dataset from scratch. Include code for module creation, compilation, training, and evaluation**

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test  = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.1,
                    verbose=2)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 9s 1us/step
Epoch 1/5


c:\Users\victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


844/844 - 8s - 9ms/step - accuracy: 0.9443 - loss: 0.1861 - val_accuracy: 0.9832 - val_loss: 0.0587
Epoch 2/5
844/844 - 7s - 8ms/step - accuracy: 0.9824 - loss: 0.0567 - val_accuracy: 0.9855 - val_loss: 0.0459
Epoch 3/5
844/844 - 6s - 8ms/step - accuracy: 0.9876 - loss: 0.0398 - val_accuracy: 0.9880 - val_loss: 0.0386
Epoch 4/5
844/844 - 7s - 8ms/step - accuracy: 0.9906 - loss: 0.0301 - val_accuracy: 0.9890 - val_loss: 0.0377
Epoch 5/5
844/844 - 7s - 8ms/step - accuracy: 0.9923 - loss: 0.0236 - val_accuracy: 0.9888 - val_loss: 0.0364
Test accuracy: 0.9890, Test loss: 0.0371


#**Question 7: Load and preprocess the CIFAR-10 dataset using Keras, and create a CNN model to classify RGB images. Show your preprocessing and architecture.**

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0
y_train = y_train.flatten()
y_test  = y_test.flatten()

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=2
)

test_loss, test_acc = model.evaluate(x_test, y_test,verbose=0)
print(f"Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 91s 1us/step
Epoch 1/10
704/704 - 31s - 44ms/step - accuracy: 0.4232 - loss: 1.5810 - val_accuracy: 0.5918 - val_loss: 1.1479
Epoch 2/10
704/704 - 30s - 42ms/step - accuracy: 0.5924 - loss: 1.1397 - val_accuracy: 0.6560 - val_loss: 0.9747
Epoch 3/10
704/704 - 30s - 42ms/step - accuracy: 0.6592 - loss: 0.9659 - val_accuracy: 0.7260 - val_loss: 0.7903
Epoch 4/10
704/704 - 30s - 43ms/step - accuracy: 0.6970 - loss: 0.8561 - val_accuracy: 0.7432 - val_loss: 0.7372
Epoch 5/10
704/704 - 30s - 42ms/step - accuracy: 0.7274 - loss: 0.7748 - val_accuracy: 0.7320 - val_loss: 0.7806
Epoch 6/10
704/704 - 29s - 42ms/step - accuracy: 0.7472 - loss: 0.7199 - val_accuracy: 0.7644 - val_loss: 0.6926
Epoch 7/10
704/704 - 29s - 42ms/step - accuracy: 0.7661 - loss: 0.6670 - val_accuracy: 0.7556 - val_loss: 0.7171
Epoch 8/10
704/704 - 29s - 41ms/step - accuracy: 0.7779 - loss: 0.6268 - val_accuracy: 0.7804 - val_loss: 0.6618
Epoch 9/10
704/704 - 30s - 42ms/step - acc

#**Question 8: Using PyTorch, write a script to define and train a CNN on the MNIST dataset. Include model definition, data loaders, training loop, and accuracy evaluation**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MNIST Data loaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

model = CNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


100.0%
100.0%
100.0%
100.0%


Epoch 1, Loss: 0.1898
Epoch 2, Loss: 0.0805
Epoch 3, Loss: 0.0620
Epoch 4, Loss: 0.0506
Epoch 5, Loss: 0.0426
Test Accuracy: 99.06%


**Question 9: Given a custom image dataset stored in a local directory, write code using Keras ImageDataGenerator to preprocess and train a CNN model**

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to your dataset
train_dir = 'datasets/train'
val_dir = 'datasets/validation'

# ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0,1]
    rotation_range=20,       # Random rotations
    width_shift_range=0.2,   # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Flip horizontally
    fill_mode='nearest'      # Fill pixels after transformation
)

val_datagen = ImageDataGenerator(
    rescale=1./255           # Only normalize for validation
)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Resize all images
    batch_size=32,
    class_mode='categorical' # For multi-class classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

# Save the trained model
model.save('custom_cnn_model.h5')


Found 294 images belonging to 2 classes.


Found 97 images belonging to 2 classes.


c:\Users\victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 706ms/step - accuracy: 0.5136 - loss: 0.8265 - val_accuracy: 0.5876 - val_loss: 0.6692
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step - accuracy: 0.6020 - loss: 0.6567 - val_accuracy: 0.6082 - val_loss: 0.6398
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 205ms/step - accuracy: 0.6088 - loss: 0.6880 - val_accuracy: 0.5979 - val_loss: 0.6472
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 203ms/step - accuracy: 0.6497 - loss: 0.6507 - val_accuracy: 0.6495 - val_loss: 0.6493
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - accuracy: 0.6667 - loss: 0.6333 - val_accuracy: 0.6495 - val_loss: 0.6345
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 205ms/step - accuracy: 0.6395 - loss: 0.6393 - val_accuracy: 0.6598 - val_loss: 0.5959
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step - accuracy: 0.6667 - loss: 0.6264 - val_accuracy: 0.6598 - val_loss: 0.6166
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 198ms/step - accuracy: 0.7041 - loss: 0.5823 - val_accuracy: 0.

**Question 10: You are working on a web application for a medical imaging startup. Your task is to build and deploy a CNN model that classifies chest X-ray images into “Normal” and “Pneumonia” categories. Describe your end-to-end approach–from data preparation and model training to deploying the model as a web app using Streamlit**

**Data: sources and preparation**

Datasets

Use a labeled chest X-ray dataset (example: “Chest X-Ray Images (Pneumonia)” often available on Kaggle / institutional dataset). If using internal clinical data, follow privacy rules (de-identify images).

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

100%|██████████| 2.29G/2.29G [07:47<00:00, 5.27MB/s] 

Extracting files...


Path to dataset files: C:\Users\victus\.cache\kagglehub\datasets\paultimothymooney\chest-xray-pneumonia\versions\2


#### **Data Preparation**

Theory:
Medical images often vary in size, quality, and orientation. Preprocessing is crucial to normalize images and augment the dataset to improve model generalization. Typical steps include resizing, normalization, and data augmentation.

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    'chest_xray/val',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


#### **Building the CNN Model**

**Theory:**

A CNN (Convolutional Neural Network) is ideal for image classification because it automatically extracts spatial features using convolutional layers.

**Common layers:**
- Conv2D: Extracts features using filters.
- MaxPooling2D: Reduces spatial dimensions.
- Flatten & Dense: Converts features into a vector for classification.
- Dropout: Prevents overfitting.

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')   # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


c:\Users\victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### **Model Training**
**Theory:**

- Binary crossentropy is used for 2-class problems.
- Adam optimizer is commonly used for faster convergence.
- Dropout reduces overfitting, especially in small medical datasets.

In [12]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

# Save the model
model.save('chest_xray_cnn.h5')


c:\Users\victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.7824 - loss: 0.4815 - val_accuracy: 0.8750 - val_loss: 0.4242
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 96s 586ms/step - accuracy: 0.8729 - loss: 0.2984 - val_accuracy: 0.8125 - val_loss: 0.5095
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 57s 352ms/step - accuracy: 0.8919 - loss: 0.2574 - val_accuracy: 0.8125 - val_loss: 0.4383
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 358ms/step - accuracy: 0.9153 - loss: 0.2151 - val_accuracy: 0.6250 - val_loss: 1.1733
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 358ms/step - accuracy: 0.9256 - loss: 0.1980 - val_accuracy: 0.8125 - val_loss: 0.5010
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 356ms/step - accuracy: 0.9248 - loss: 0.1906 - val_accuracy: 0.6875 - val_loss: 0.6862
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 354ms/step - accuracy: 0.9302 - loss: 0.1816 - val_accuracy: 0.8750 - val_loss: 0.2956
Epoch 8/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 55s 338ms/step - accuracy: 0.9438 - loss: 0.1

#### **Model Evaluation**
- Plot accuracy and loss curves for visual inspection.
- Check for misclassifications and confusion matrix for deeper insight.

In [13]:
test_generator = val_datagen.flow_from_directory(
    'chest_xray/test',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Found 624 images belonging to 2 classes.
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 429ms/step - accuracy: 0.8750 - loss: 0.3717
Test Accuracy: 87.50%


#### **Deploying as a Web App using Streamlit**

**Theory:**

Streamlit is a Python framework for creating interactive web apps easily. Users can upload X-ray images, and the app displays the model’s prediction.

Streamlit App Code (app.py):

In [14]:
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load trained model
model = load_model('chest_xray_cnn.h5')

st.title("Chest X-ray Pneumonia Classifier")

uploaded_file = st.file_uploader("Upload a chest X-ray image", type=["png","jpg","jpeg"])

if uploaded_file:
    img = image.load_img(uploaded_file, target_size=(128,128))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)[0][0]
    
    if prediction < 0.5:
        st.success("Prediction: Normal")
    else:
        st.error("Prediction: Pneumonia")


2025-10-03 15:29:09.491 
  command:

    streamlit run c:\Users\victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
